In [ ]:
# Cargar los datos y obtener el modelo
from data_and_model_construction.data import create_data_dict
from data_and_model_construction.common_model import create_model, solve_model
from plot_kind.funcional_vs_disp import Funcional

data = create_data_dict()
mdl, production_vars, products = create_model(data)
solve_model(mdl, production_vars, products)

Model: produccion_pullover
 - number of variables: 5
   - binary=0, integer=0, continuous=5
 - number of constraints: 15
   - linear=15
 - parameters: defaults
 - objective: maximize
 - problem type is: LP
* Production model solved with objective: 550
* Total benefit=550
Production of A: 0
Production of B: 16.666666666666664
Production of B1: 13.333333333333332
Production of B2: 3.333333333333332
Production of C: 16.666666666666668


In [ ]:
# Usage of the iterate_over_rhs method
constraint_name = 'DemandMin_B'  # The name of the constraint to analyze
funcional = Funcional(mdl, products, production_vars)
real_rhs_value, rhs_values, objective_values = funcional.iterate(constraint_name)

---Initial lower bound: -1e+20
---
- Adjusting RHS to: -1e+20
* Production model solved with objective: 550
* Total benefit=550
Production of A: 0
Production of B: 16.666666666666664
Production of B1: 13.333333333333332
Production of B2: 3.333333333333332
Production of C: 16.666666666666668
---Initial upper bound: 16.666666666666664
---
- Adjusting RHS to: 16.666666666666664
* Production model solved with objective: 550
* Total benefit=550
Production of A: 0
Production of B: 16.666666666666664
Production of B1: 13.333333333333332
Production of B2: 3.333333333333332
Production of C: 16.666666666666668
---
- Adjusting RHS to: 16.676666666666666
* Production model solved with objective: 549.97
* Total benefit=549.97
Production of A: 0
Production of B: 16.676666666666666
Production of B1: 13.333333333333332
Production of B2: 3.3433333333333337
Production of C: 16.656666666666666
---
- Adjusting RHS to: 20.0
* Production model solved with objective: 540
* Total benefit=540
Production of A: 

In [ ]:
funcional.plot(constraint_name, "[un/semana]", "[$/semana]")

[debug] pairs [(10, 550.0), (-1e+20, 550.0), (16.666666666666664, 550.0), (20.0, 540.0)]
[debug] pairs [(-1e+20, 550.0), (10, 550.0), (16.666666666666664, 550.0), (20.0, 540.0)]
[debug] new_rhs [-1e+20, 10, 16.666666666666664, 20.0]
[debug] new_dual [550.0, 550.0, 550.0, 540.0]
[debug] current_x_value: 10


<Figure size 640x480 with 1 Axes>

<Figure size 2000x1000 with 0 Axes>